In [3]:

from os import listdir
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

groups = ["Cr","In","Pa","PS","RS","Sc"]
data = [[] for i in range(len(groups))]

for f in listdir("data"):
    image = np.reshape(cv2.equalizeHist(cv2.imread("data/"+f,0)),[40000])
    data[groups.index(f[0:2])].append(image)
    

from pylearn2.models.model import Model
from pylearn2.space import VectorSpace
from pylearn2.models.mlp import PretrainedLayer
import theano
from theano import tensor
from pylearn2.utils import sharedX
import theano.tensor as T

from __future__ import print_function

__author__ = "Li Yao"
"""
See readme.txt

A small example of how to glue shining features of pylearn2 together
to train models layer by layer.
"""

MAX_EPOCHS_UNSUPERVISED = 1
MAX_EPOCHS_SUPERVISED = 2

from pylearn2.config import yaml_parse
from pylearn2.corruption import BinomialCorruptor
from pylearn2.corruption import GaussianCorruptor
from pylearn2.costs.mlp import Default
from pylearn2.models.autoencoder import Autoencoder, DenoisingAutoencoder
from pylearn2.models.rbm import GaussianBinaryRBM
from pylearn2.models.DRM import DeepEncoder
from pylearn2.models.softmax_regression import SoftmaxRegression
from pylearn2.training_algorithms.sgd import SGD
from pylearn2.training_algorithms.sgd import AnnealedLearningRate
from pylearn2.training_algorithms.sgd import OneOverEpoch
from pylearn2.costs.autoencoder import MeanSquaredReconstructionError
from pylearn2.termination_criteria import EpochCounter
from pylearn2.datasets.dense_design_matrix import DenseDesignMatrix
from pylearn2.energy_functions.rbm_energy import GRBM_Type_1
from pylearn2.blocks import StackedBlocks
from pylearn2.datasets.transformer_dataset import TransformerDataset
from pylearn2.costs.ebm_estimation import SMD
from pylearn2.costs.DRM import DRMCost
from pylearn2.training_algorithms.sgd import MonitorBasedLRAdjuster
from pylearn2.train import Train
from optparse import OptionParser
from pylearn2.datasets.honda import load_data
import numpy
import copy
MAX_EPOCHS_UNSUPERVISED = 1
MAX_EPOCHS_SUPERVISED = 2



def get_autoencoder(structure):
    n_input, n_output = structure
    config = {
        'nhid': n_output,
        'nvis': n_input,
        'tied_weights': True,
        'act_enc': 'sigmoid',
        'act_dec': 'sigmoid',
        'irange': 0.001,
    }
    return Autoencoder(**config)

def get_denoising_autoencoder(structure):
    n_input, n_output = structure
    curruptor = BinomialCorruptor(corruption_level=0.5)
    config = {
        'corruptor': curruptor,
        'nhid': n_output,
        'nvis': n_input,
        'tied_weights': True,
        'act_enc': 'sigmoid',
        'act_dec': 'sigmoid',
        'irange': 0.001,
    }
    return DenoisingAutoencoder(**config)

def get_grbm(structure):
    n_input, n_output = structure
    config = {
        'nvis': n_input,
        'nhid': n_output,
        "irange" : 0.05,
        "energy_function_class" : GRBM_Type_1,
        "learn_sigma" : True,
        "init_sigma" : .4,
        "init_bias_hid" : -2.,
        "mean_vis" : False,
        "sigma_lr_scale" : 1e-3
        }

    return GaussianBinaryRBM(**config)

def get_logistic_regressor(structure):
    n_input, n_output = structure

    layer = SoftmaxRegression(n_classes=n_output, irange=0.02, nvis=n_input)

    return layer

def get_layer_trainer_logistic(layer, trainset):
    # configs on sgd

    config = {'learning_rate': 0.1,
              'cost' : Default(),
              'batch_size': 10,
              'monitoring_batches': 10,
              'monitoring_dataset': trainset,
              'termination_criterion': EpochCounter(max_epochs=50),
              'update_callbacks': None
              }

    train_algo = SGD(**config)
    model = layer
    return Train(model = model,
            dataset = trainset,
            algorithm = train_algo,
            extensions = None)

def get_layer_trainer_sgd_autoencoder(layer, trainset):
    # configs on sgd
    train_algo = SGD(
            learning_rate = 2*1e-3,
              cost =  DRMCost(),
              batch_size =  1,
              monitoring_batches = 5,
              monitoring_dataset =  trainset,
              termination_criterion = EpochCounter(max_epochs=5),
              update_callbacks =  None
              )

    model = layer
    extensions = None
    return Train(model = model,
            algorithm = train_algo,
            extensions = extensions,
            save_path='my_model.pkl'
            ,save_freq=1,
            dataset = trainset)

def get_layer_trainer_sgd_rbm(layer, trainset,epochs):
    train_algo = SGD(
        learning_rate = 1e-3,
        batch_size =  1,
        #"batches_per_iter" : 2000,
        monitoring_batches =  5,
        monitoring_dataset =  trainset,
        cost = SMD(corruptor=GaussianCorruptor(stdev=0.4)),
        termination_criterion =  EpochCounter(max_epochs=epochs),
        )
    model = layer
    extensions = [MonitorBasedLRAdjuster()]
    return Train(model = model, algorithm = train_algo,
                 save_path='grbm.pkl',save_freq=1,
                 extensions = extensions, dataset = trainset)



def encoder_model(trainset):
    design_matrix = trainset.get_design_matrix()
    n_input = design_matrix.shape[1]
    layers = []
    structure = [[n_input, 500],[500, n_input]]
    # layer 0: gaussianRBM
    layers.append(get_grbm(structure[0]))
    #layers.append(get_grbm(structure[1]))
    #layers.append(get_grbm(structure[2]))
    
    #layers.append(get_grbm(structure[2]))
    # layer 3: logistic regression used in supervised training                                                                                                                                               
#    layers.append(get_logistic_regressor(structure[1]))
    
    trainset = [ trainset #,
                #TransformerDataset( raw = trainset, transformer = layers[0] ),
                #TransformerDataset( raw = trainset, transformer = StackedBlocks( layers[0:2] )),
               ]

    # construct layer trainers
    layer_trainers = []
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[0], trainset[0],5))
    #layer_trainers.append(get_layer_trainer_sgd_rbm(layers[1], trainset[1],20))
    #layer_trainers.append(get_layer_trainer_sgd_rbm(layers[2], trainset[2],20))

    #layer_trainers.append(get_layer_trainer_sgd_rbm(layers[1], trainset[1],5))
    #layer_trainers.append(get_layer_trainer_sgd_rbm(layers[2], trainset[2],5))
    #layer_trainers.append(get_layer_trainer_logistic(layers[1], trainset[1]))
    # unsupervised pretraining
    for i, layer_trainer in enumerate(layer_trainers):
        print('-----------------------------------')
        print(' Unsupervised training layer %d, %s'%(i, layers[i].__class__))
        print('-----------------------------------')
        layer_trainer.main_loop()
    encoder = DeepEncoder(layers)
    print('\n')
    print('------------------------------------------------------')
    print(' Unsupervised training done! Start Fine-tuing training...')
    print('------------------------------------------------------')
    print('\n')
    # supervised training
    #layer_trainers[-1].main_loop()
    trainer = get_layer_trainer_sgd_autoencoder(encoder,trainset[0])
    trainer.main_loop()
    return encoder



In [4]:
def random_split(data):
    train = []
    test = np.asarray([])
    labels = []
    for i in range(len(groups)):
        labels = labels + [i]*150
        current_set = data[i]
        np.random.shuffle(current_set)
        if(len(test)==0):
#            train = current_set[:150]
            test = current_set[150:]
        else:
#            train = np.vstack([train,current_set[:150]])
            test = np.vstack([test,current_set[150:]])
        train.append(current_set[:150])
        
    return train,test,labels

train,test,label = random_split(data)

encoder_models = []
for k,train in enumerate(train):
    t = copy.copy(np.asarray(train))
    encoder_models.append(encoder_model(DenseDesignMatrix(X=t,y=None)))

#t = copy.copy(np.asarray(train[2]))
#encoder_models.append(encoder_model(DenseDesignMatrix(X=t,y=None)))

-----------------------------------
 Unsupervised training layer 0, <class 'pylearn2.models.rbm.GaussianBinaryRBM'>
-----------------------------------
Parameter and initial learning rate summary:
	W: 0.0010000000475
	bias_vis: 0.0010000000475
	bias_hid: 0.0010000000475
	sigma_driver: 0.0010000000475
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 6.127932 seconds
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.030340 seconds
Monitored channels: 
	bias_hid_max
	bias_hid_mean
	bias_hid_min
	bias_vis_max
	bias_vis_mean
	bias_vis_min
	h_max
	h_mean
	h_min
	learning_rate
	objective
	reconstruction_error
	total_seconds_last_epoch
	training_seconds_this_epoch
Compiling accum...
graph size: 91
Compiling accum done. Time elapsed: 0.612159 seconds
Monitoring step:
	Epochs seen: 0
	Batches seen: 0
	Examples seen: 0
	bias_hid_max: -2.0
	bias_hid_mean: -2.00000023842
	bias_hid_min: -2.0
	bias_vis_max: 0.0
	bias_vis_mean: 0.0
	bias_vis_min: 0.0
	h

MemoryError: Error allocating 80000000 bytes of device memory (out of memory).
Apply node that caused the error: GpuAlloc{memset_0=True}(CudaNdarrayConstant{0.0}, Shape_i{1}.0, Shape_i{1}.0)
Inputs types: [CudaNdarrayType(float32, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), ()]
Inputs strides: [(), (), ()]
Inputs values: [<CudaNdarray object at 0x7f4e888d6670>, array(40000), array(500)]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [8]:

t = copy.copy(np.asarray(train[3]))
encoder_models.append(encoder_model(DenseDesignMatrix(X=t,y=None)))

-----------------------------------
 Unsupervised training layer 0, <class 'pylearn2.models.rbm.GaussianBinaryRBM'>
-----------------------------------
Parameter and initial learning rate summary:
	W: 0.0010000000475
	bias_vis: 0.0010000000475
	bias_hid: 0.0010000000475
	sigma_driver: 0.0010000000475
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 0.275843 seconds
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.016108 seconds
Monitored channels: 
	bias_hid_max
	bias_hid_mean
	bias_hid_min
	bias_vis_max
	bias_vis_mean
	bias_vis_min
	h_max
	h_mean
	h_min
	learning_rate
	objective
	reconstruction_error
	total_seconds_last_epoch
	training_seconds_this_epoch
Compiling accum...
graph size: 187
Compiling accum done. Time elapsed: 0.368952 seconds
Monitoring step:
	Epochs seen: 0
	Batches seen: 0
	Examples seen: 0
	bias_hid_max: -2.00000023842
	bias_hid_mean: -2.00000023842
	bias_hid_min: -2.00000023842
	bias_vis_max: 0.0
	bias_vis_mean: 0.0


In [11]:
encoder_models[0]

In [4]:
import theano
X = encoder_models[0].get_input_space().make_theano_batch(batch_size=1000)
output_pool = []

for encoder in encoder_models:
    X_hat = encoder.reconstruct(X)
    loss_data = ((X - X_hat)**2).sum(axis=1)
    output_pool.append(loss_data)

f = theano.function([X],output_pool)


def find_predictions(data):
    reconstruct_result = f(data)
    predictions = []
    votes_bucket = [0 for i in range(len(reconstruct_result))]
    for i in range(len(reconstruct_result[0])):
        reconstruct_err_values = []
        for j in range(len(reconstruct_result)):
            reconstruct_err_values.append(reconstruct_result[j][i])
        minimal_reconstruction = min(reconstruct_err_values)
        prediction = reconstruct_err_values.index(minimal_reconstruction) + 1
        predictions.append(prediction)
        votes_bucket[prediction-1] += 1
    
    maximal_predicted = max(votes_bucket)
    predicted_class = votes_bucket.index(maximal_predicted) + 1
    return predicted_class,predictions

total_number = float(len(test_label))
correct_count = 0
for k,true_label in enumerate(test_label):
    pred,_ = find_predictions(copy.copy(test_data[k][0]))
    if(pred==true_label):
        correct_count += 1
print(correct_count/total_number)

NameError: name 'test_label' is not defined

In [10]:
import cPickle
ff = open("trial_train.pkl","wb")
cPickle.dump(encoder_models,ff)
ff.close()

In [1]:
import cPickle
ff = open("trial_train.pkl","rb")
encoder_models = cPickle.load(ff)

MemoryError: ('Error allocating 80000000 bytes of device memory (out of memory).', <function CudaNdarray_unpickler at 0x7f94a6756320>, (array([[-0.01937678,  0.01412687,  0.00724369, ..., -0.01132945,
         0.01676371,  0.07814937],
       [ 0.0021261 ,  0.01960999,  0.02260183, ..., -0.01281641,
        -0.03500316,  0.00452814],
       [-0.03906006, -0.01323096,  0.01107634, ...,  0.01785509,
        -0.03305104,  0.06571223],
       ..., 
       [ 0.03632317,  0.03930932,  0.05246469, ..., -0.0083575 ,
        -0.0417932 ,  0.08342919],
       [ 0.03506872,  0.06747298,  0.0748592 , ...,  0.02429776,
        -0.01334724,  0.11005408],
       [ 0.0028928 ,  0.0510247 ,  0.12638801, ..., -0.0455821 ,
        -0.03785853,  0.07617415]], dtype=float32),))

In [7]:
test

NameError: name 'test' is not defined